In [1]:
ids1 = [534384996062, 41807701711, 555527716960,549360164208]

In [2]:
from taobao_crawler import ProductCrawler
crawler = ProductCrawler()

In [3]:
result = crawler.taobao_crawler(ids1, "user_id", "user_pw")

프록시 가져옴
로그인 완료
itemid : 549360164208 가격 정보 가져옴
itemid : 555527716960 가격 정보 가져옴
itemid : 41807701711 가격 정보 가져옴
itemid : 534384996062 가격 정보 가져옴


In [4]:
result

[{'item_id': 549360164208,
  'title': '新款帆布男士手包大容量牛津布手拿包时尚休闲夹包迷彩手机包尼龙',
  'og_price': '78.00',
  'promo_price': '22.00',
  'item_title': '新款帆布男士手包大容量牛津布手拿包时尚休闲夹包迷彩手机包尼龙',
  'options': [{'option_title': '颜色分类',
    'option_details': ['迷彩蓝', '迷彩黑', '纯黑色'],
    'option_image_urls': ['gd2.alicdn.com/imgextra/i4/1973935423/TB2n5vDpt4opuFjSZFLXXX8mXXa_!!1973935423.jpg',
     'gd4.alicdn.com/imgextra/i3/1973935423/TB2YMSrmYBkpuFjy1zkXXbSpFXa_!!1973935423.jpg',
     'gd4.alicdn.com/imgextra/i3/1973935423/TB2QfpPclDH8KJjy1zeXXXjepXa_!!1973935423.jpg']}]},
 {'item_id': 555527716960,
  'title': '天然药珀手串108颗佛珠天意菩提子念珠男女款莲花菩提根手链项链',
  'og_price': '52.00-128.00',
  'promo_price': '50.00-126.00',
  'item_title': '天然药珀手串108颗佛珠天意菩提子念珠男女款莲花菩提根手链项链',
  'options': [{'option_title': '尺寸',
    'option_details': ['10mm', '6mm', '8mm'],
    'option_image_urls': []},
   {'option_title': '颜色分类',
    'option_details': ['2圈款（16cm手围）',
     '3圈款（16cm手围）',
     '2圈款（17cm手围）',
     '3圈款（17cm手围）',
     '108颗弹力线款',
    

## 프록시 가져오는 함수

In [34]:
from selenium import webdriver
from selenium.webdriver.chrome.options import DesiredCapabilities
from selenium.webdriver.common.proxy import Proxy, ProxyType

import time


co = webdriver.ChromeOptions()
co.add_argument("log-level=3")
co.add_argument("--headless")

def get_proxies(co=co):
    driver = webdriver.Chrome('./chromedriver',chrome_options=co)
    driver.get("https://free-proxy-list.net/")

    PROXIES = []
    proxies = driver.find_elements_by_css_selector("tr[role='row']")
    for p in proxies:
        result = p.text.split(" ")

        if result[-1] == "yes":
            PROXIES.append(result[0]+":"+result[1])

    driver.close()
    return PROXIES



In [35]:
proxies = get_proxies()

In [36]:
proxies

['196.1.184.6:52963',
 '86.125.112.230:57373',
 '75.109.249.111:1111',
 '154.66.241.27:52004',
 '125.26.7.124:61642']

# 로그인창 입력

In [364]:
from selenium import webdriver
driver = webdriver.Chrome('./chromedriver')
driver.get("https://detail.tmall.com/item.htm?id=540619456825")
try:
    alert = driver.switch_to.alert
    alert.accept()
except:
    time.sleep(1)

In [5]:
from selenium import webdriver
driver = webdriver.Chrome('./chromedriver')
driver.get("https://item.taobao.com/item.htm?id=586855541639")
try:
    alert = driver.switch_to.alert
    alert.accept()
except:
    time.sleep(1)

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

driver = webdriver.Chrome('./chromedriver')
driver.get("https://item.taobao.com/item.htm?id=586855541639")
try:
    alert = driver.switch_to.alert
    alert.accept()
except:
    time.sleep(1)

driver.switch_to_frame("sufei-dialog-content")  
id_box = driver.find_element_by_id('fm-login-id')
pw_box = driver.find_element_by_id('fm-login-password')
swipe_btn = driver.find_element_by_id('nc_1_n1z')

webdriver.ActionChains(driver).send_keys_to_element(id_box, 'yoohowon90').send_keys_to_element(pw_box,'ghdnjs77').perform()
time.sleep(1)

try:
    element = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "nc_1_n1z"))
    )

finally:
    webdriver.ActionChains(driver).click_and_hold(swipe_btn).move_to_element_with_offset(swipe_btn, 300,10).release().perform()
time.sleep(1)

login_btn = driver.find_element_by_class_name('fm-btn').find_element_by_tag_name('button')
webdriver.ActionChains(driver).click(login_btn).perform()


driver.switch_to.default_content()

try:
    element = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "J_PromoPriceNum"))
    )

finally:
    result = {}
    result['title'] = driver.find_element_by_xpath('//*[@id="J_Title"]/h3').text
    result['promo_price'] =  "¥" + driver.find_element_by_xpath('//*[@id="J_PromoPriceNum"]').text
    result['og_price'] = driver.find_element_by_xpath('//*[@id="J_StrPrice"]').text


In [371]:
result

{'title': '唐麦 T3耳机 入耳式 苹果安卓华为小米通用手机K歌有线耳塞式耳麦',
 'promo_price': '¥99.00-117.00',
 'og_price': '¥109.00-129.00'}

## 연속적으로 여러번 요청하면 배너 뜨는거 해결 해야 함

# 쿠키 저장

In [143]:
c = driver.get_cookies()


In [146]:
import pickle
f = open('cookie_taobao.dat', 'rb')
cookies = pickle.load(f)

In [154]:
import pickle
# tmall 쿠키 백업
c = driver.get_cookies()
with open('cookie_taobao.dat', 'wb') as f:
    pickle.dump(c, f)

In [160]:
import pickle
# taobao 쿠키 백업
c = driver.get_cookies()
with open('cookie_tamll.dat', 'wb') as f:
    pickle.dump(c, f)

# selenium

In [124]:
def crawler(itemid):
    
    from selenium import webdriver
    from fake_useragent import UserAgent

    # userAgent 설정
    ua = UserAgent(verify_ssl=False)
    userAgent = ua.random
    headers = {
        'User-Agent': userAgent
    }
    options = webdriver.ChromeOptions()
    options.add_argument(userAgent)


    driver = webdriver.Chrome('./chromedriver',options = options)
    
        
    driver.get("https://item.taobao.com/item.htm?id={}".format(itemid))

    # alert 창 제어
    try:
        alert = driver.switch_to_alert()
        alert.accept()
        current_url = driver.current_url
    except:
        current_url = driver.current_url
    driver.quit()
    
    def tmall_crawler(itemid):
        from selenium import webdriver
        from scrapy.http import TextResponse     
        from fake_useragent import UserAgent

        import requests
        import pickle

        driver = webdriver.Chrome('./chromedriver')
        driver.get("https://www.taobao.com")


        f = open('cookie_taobao.dat', 'rb') 
        cookies = pickle.load(f) 
        for cookie in cookies:
            driver.add_cookie(cookie)

        driver.get("https://item.taobao.com/item.htm?id={}".format(itemId))   

        result = {}
        result['item_id'] = itemId

        # 할인 가격
        try:
            result['promo_price'] = driver.find_element_by_xpath('//*[@id="J_PromoPrice"]/dd/div/span').text
        except:
            result['promo_price'] = '할인 가격 없음'

        driver.quit()

        url = "https://item.taobao.com/item.htm?id={}".format(itemId)

        ua = UserAgent(verify_ssl=False)
        userAgent = ua.random

        headers = {
            'User-Agent': userAgent
        }
        response = requests.get(url=url, headers=headers)
        req = TextResponse(response.url, body=response.text, encoding="utf-8")

        # item_title
        result['item_title'] = req.xpath('//*[@id="J_DetailMeta"]/div[@class="tm-clear"]/div[@class="tb-property"]/div/div[@class="tb-detail-hd"]/h1/text()').extract_first().strip()

        option_path = req.xpath('//*[@id="J_DetailMeta"]/div/div/div/div[@class="tb-key"]/div[@class="tb-skin"]/div[@class="tb-sku"]/dl[contains(@class, "tm-sale-prop") and contains(@class, "tb-prop") and contains(@class, "tm-clear")]')

        option_ls = []
        for path in option_path:
            option = {}
            option['option_title'] = path.xpath('dt/text()').extract_first()
            option['option_detail'] = path.xpath('dd/ul/li/a/span/text()').extract()
            image_elements = path.xpath('dd/ul[contains(@class, "tb-img")]/li/a')
            option['option_image_urls'] = [element.xpath('@style').extract_first()[19:-40] for element in image_elements]
            option_ls.append(option)
            result['options'] = option_ls
        
        return result

    def taobao_crawler(itemId):
        # 패키지 임포트
        from selenium import webdriver
        from scrapy.http import TextResponse     
        from fake_useragent import UserAgent
        import pickle

        import requests

        driver = webdriver.Chrome('./chromedriver')
        driver.get("https://www.taobao.com")

        f = open('cookie_taobao.dat', 'rb') 
        cookies = pickle.load(f) 
        for cookie in cookies:
            driver.add_cookie(cookie)


        driver.get("https://item.taobao.com/item.htm?id={}".format(itemId))   

        result = {}
        result['item_id'] = itemId

        # 할인 가격
        try:
            result['promo_price'] = driver.find_element_by_css_selector('#J_PromoPriceNum').text
        except:
            result['promo_price'] = '할인 가격 없음'

        driver.quit()

        ua = UserAgent(verify_ssl=False)
        userAgent = ua.random

        headers = {
            'User-Agent': userAgent
        }

        url = "https://item.taobao.com/item.htm?id={}".format(itemId)

        # https://stackoverflow.com/questions/32910093/python-requests-gets-tlsv1-alert-internal-err
        requests.packages.urllib3.util.ssl_.DEFAULT_CIPHERS = 'DH+AESGCM:ECDH+AES256:DH+AES256:ECDH+AES128:DH+AES:ECDH+HIGH:DH+HIGH:ECDH+3DES:DH+3DES:RSA+AESGCM:RSA+AES:RSA+HIGH:RSA+3DES:!aNULL:!eNULL:!MD5'

        response = requests.get(url=url, headers=headers)
        req = TextResponse(response.url, body=response.text, encoding="utf-8")
        result['item_name'] = req.xpath('//*[@id="J_Title"]/h3/@data-title').extract_first()

        option_title = req.xpath('//*[@id="J_isku"]/div/dl/dd/ul/@data-property').extract()
        r = []
        for title in option_title:
            j = {}
            j['option_title'] = title
            j['option_details'] = req.xpath('//*[@id="J_isku"]/div/dl/dd/ul[@data-property="{}"]/li/a/span/text()'.format(title)).extract()
            j['option_image_urls'] = [path[17:-29] for path in req.xpath('//*[@id="J_isku"]/div/dl/dd/ul[@data-property="{}"]/li/a/@style'.format(title)).extract()]
            r.append(j)

        result['options'] = r

        return result

    
    if "tmall" in current_url:
        return tmall_crawler(itemid)
    else:
        return taobao_crawler(itemid)

# test

In [126]:
tmall_item = crawler(594650607691)

In [127]:
tmall_item

{'item_id': 594650607691,
 'promo_price': '118.00',
 'item_title': '啄木鸟 丝光棉夏季男士短袖翻领t恤口袋有领中年爸爸装Polo衫大码',
 'options': [{'option_title': '尺码',
   'option_detail': ['165/S',
    '170/M',
    '175/L',
    '180/XL',
    '185/XXL',
    '190/3XL'],
   'option_image_urls': []},
  {'option_title': '颜色',
   'option_detail': ['ZMN7102-宝蓝', 'ZMN7102-白色', 'ZMN7102-酒红色', 'ZMN7102-灰色'],
   'option_image_urls': ['g.alicdn.com/imgextra/i3/2997589458/O1CN01K4KHx82JjoWAy7mTB_!!299758',
    'g.alicdn.com/imgextra/i2/2997589458/O1CN01FCMild2JjoWA7LyYw_!!299758',
    'g.alicdn.com/imgextra/i2/2997589458/O1CN01zwiVj62JjoW7iZYMd_!!299758',
    'g.alicdn.com/imgextra/i2/2997589458/O1CN01A2jmnN2JjoW9ExK8X_!!299758']}]}

In [128]:
taobao_item = crawler(524341898973)

In [129]:
taobao_item

{'item_id': 524341898973,
 'promo_price': '199.00',
 'item_name': '阎双女包双肩包女个性时尚韩版大容量花包复古风网红学生书包2093',
 'options': [{'option_title': '颜色分类',
   'option_details': ['荷花',
    '吊染花',
    '绿章',
    '深灰色',
    '黑白色',
    '红咖啡',
    '毕加索',
    '荧光绿',
    '莲藕',
    '春天棉',
    '藏码',
    '抽图',
    '灰色',
    '蓝黑色',
    '军绿色',
    '东北红',
    '湖蓝色',
    '酒红色',
    '黑色',
    '开花',
    '东巴蛙',
    '宝蓝色',
    '马勺',
    '米白色',
    '紫红色',
    '品韵',
    '老尘土',
    '乳白色',
    '花色',
    '西瓜红',
    '桔色',
    '银色',
    '天蓝色',
    '浅黄色',
    '翠绿色',
    '浅灰色',
    '深紫色',
    '粉红色',
    '季春',
    '绽放',
    '姜黄色',
    '黄色',
    '卡其色',
    '栗色',
    '黑白',
    '玫红色',
    '白色',
    '浅蓝色',
    '青色',
    '驼色',
    '巧克力色',
    '红色',
    '浅棕色',
    '褐色',
    '浅紫色',
    '孔雀蓝',
    '蓝色',
    '香槟色',
    '紫色',
    '明黄色',
    '深棕色',
    '荧光黄',
    '藏青色',
    '咖啡色',
    '深卡其布色',
    '深蓝色',
    '紫蓝色',
    '紫罗兰',
    '米驼色',
    '杏色',
    '透明',
    '墨绿色',
    '桔红色',
    '柠檬黄',
    '藕色',
    '棕咖',
    '蓝莓色',
    '浅绿色'],
   'optio

# tmall request 활용

In [118]:
from selenium import webdriver
from scrapy.http import TextResponse     
from fake_useragent import UserAgent

import requests
import pickle

driver = webdriver.Chrome('./chromedriver')
driver.get("https://www.taobao.com")


f = open('cookie_taobao.dat', 'rb') 
cookies = pickle.load(f) 
for cookie in cookies:
    driver.add_cookie(cookie)

driver.get("https://item.taobao.com/item.htm?id={}".format(itemId))   

result = {}
result['item_id'] = itemId

# 할인 가격
try:
    result['promo_price'] = driver.find_element_by_xpath('//*[@id="J_PromoPrice"]/dd/div/span').text
except:
    result['promo_price'] = '할인 가격 없음'

driver.quit()

url = "https://item.taobao.com/item.htm?id={}".format(itemId)

ua = UserAgent(verify_ssl=False)
userAgent = ua.random

headers = {
    'User-Agent': userAgent
}
response = requests.get(url=url, headers=headers)
req = TextResponse(response.url, body=response.text, encoding="utf-8")

# item_title
result['item_title'] = req.xpath('//*[@id="J_DetailMeta"]/div[@class="tm-clear"]/div[@class="tb-property"]/div/div[@class="tb-detail-hd"]/h1/text()').extract_first().strip()

option_path = req.xpath('//*[@id="J_DetailMeta"]/div/div/div/div[@class="tb-key"]/div[@class="tb-skin"]/div[@class="tb-sku"]/dl[contains(@class, "tm-sale-prop") and contains(@class, "tb-prop") and contains(@class, "tm-clear")]')

option_ls = []
for path in option_path:
    option = {}
    option['option_title'] = path.xpath('dt/text()').extract_first()
    option['option_detail'] = path.xpath('dd/ul/li/a/span/text()').extract()
    image_elements = path.xpath('dd/ul[contains(@class, "tb-img")]/li/a')
    option['option_image_urls'] = [element.xpath('@style').extract_first()[19:-40] for element in image_elements]
    option_ls.append(option)
    result['options'] = option_ls

In [123]:
result

{'item_id': 594650607691,
 'promo_price': '118.00',
 'item_title': '啄木鸟 丝光棉夏季男士短袖翻领t恤口袋有领中年爸爸装Polo衫大码',
 'options': [{'option_title': '尺码',
   'option_detail': ['165/S',
    '170/M',
    '175/L',
    '180/XL',
    '185/XXL',
    '190/3XL'],
   'option_image_urls': []},
  {'option_title': '颜色',
   'option_detail': ['ZMN7102-宝蓝', 'ZMN7102-白色', 'ZMN7102-酒红色', 'ZMN7102-灰色'],
   'option_image_urls': ['g.alicdn.com/imgextra/i3/2997589458/O1CN01K4KHx82JjoWAy7mTB_!!299758',
    'g.alicdn.com/imgextra/i2/2997589458/O1CN01FCMild2JjoWA7LyYw_!!299758',
    'g.alicdn.com/imgextra/i2/2997589458/O1CN01zwiVj62JjoW7iZYMd_!!299758',
    'g.alicdn.com/imgextra/i2/2997589458/O1CN01A2jmnN2JjoW9ExK8X_!!299758']}]}

In [1]:
# tmall
itemid = "38012241391"

In [4]:
# taobao
itemid = "524341898973"

In [5]:
from selenium import webdriver
from fake_useragent import UserAgent

# userAgent 설정
ua = UserAgent(verify_ssl=False)
userAgent = ua.random
headers = {
    'User-Agent': userAgent
}
options = webdriver.ChromeOptions()
options.add_argument(userAgent)


driver = webdriver.Chrome('./chromedriver',options = options)
driver.get("https://item.taobao.com/item.htm?id={}".format(itemid))

# alert 창 제어
try:
    alert = driver.switch_to_alert()
    alert.accept()
    current_url = driver.current_url
except:
    current_url = driver.current_url

    
driver.quit()

In [4]:
def tmall_crawler(itemid, driver):
    import pickle
    # 쿠키 복원
    f = open('cookie_taobao.dat', 'rb') 
    cookies = pickle.load(f) 
    for c in cookies: 
        driver.add_cookie(c)
    driver.implicitly_wait(1)
    driver.get("https://item.taobao.com/item.htm?id={}".format(itemid))
    
    result = {}
    result['item_id'] = itemid
    result['item_title'] = driver.find_element_by_xpath('//*[@id="J_DetailMeta"]/div[1]/div[1]/div/div[1]/h1').text
    result['og_price'] = driver.find_element_by_xpath('//*[@id="J_StrPriceModBox"]/dd/span').text
    result['promo_price'] = driver.find_element_by_xpath('//*[@id="J_PromoPrice"]/dd/div/span').text
    option_path = driver.find_elements_by_xpath('//*[@id="J_DetailMeta"]/div/div/div/div[@class="tb-key"]/div[@class="tb-skin"]/div[@class="tb-sku"]/dl[contains(@class, "tm-sale-prop") and contains(@class, "tb-prop") and contains(@class, "tm-clear")]')
    option_ls = []
    for path in option_path:
        option = {}
        option['option_title'] = path.find_element_by_xpath('dt').text
        option['oprion_detail'] = path.find_element_by_xpath('dd/ul').text.split('\n')
        image_elements = path.find_elements_by_xpath('dd/ul[contains(@class, "tb-img")]/li/a')
        option['option_image_urls'] = [element.get_attribute('style')[19:-40] for element in image_elements]
        option_ls.append(option)
        result['options'] = option_ls
        driver.quit()
    return result


In [5]:
def taobao_crawler(itemId,driver):
    result = {}
    result['itemID'] = itemId
    
    # 패키지 임포트
    from selenium import webdriver
    from scrapy.http import TextResponse     
    from fake_useragent import UserAgent
    
    import pickle
    import requests
    import re
    
#     driver = webdriver.Chrome('./chromedriver')
#     driver.get("https://www.taobao.com")
    
    # 쿠키 복원
    f = open('cookie_taobao.dat', 'rb') 
    cookies = pickle.load(f) 
    for cookie in cookies:
        driver.add_cookie(cookie)
        
    driver.get("https://item.taobao.com/item.htm?id={}".format(itemId))   

    # 할인 가격
    try:
        result['promo_price'] = driver.find_element_by_css_selector('#J_PromoPriceNum').text
    except:
        result['promo_price'] = '할인 가격 없음'
        
    driver.quit()
    
    ua = UserAgent(verify_ssl=False)
    userAgent = ua.random
    
    headers = {
        'User-Agent': userAgent
    }

    url = "https://item.taobao.com/item.htm?id={}".format(itemId)

    # https://stackoverflow.com/questions/32910093/python-requests-gets-tlsv1-alert-internal-err
    requests.packages.urllib3.util.ssl_.DEFAULT_CIPHERS = 'DH+AESGCM:ECDH+AES256:DH+AES256:ECDH+AES128:DH+AES:ECDH+HIGH:DH+HIGH:ECDH+3DES:DH+3DES:RSA+AESGCM:RSA+AES:RSA+HIGH:RSA+3DES:!aNULL:!eNULL:!MD5'

    response = requests.get(url=url, headers=headers)
    req = TextResponse(response.url, body=response.text, encoding="utf-8")
    
    # Taobao
    if len(req.xpath('//div[contains(@class, "tm-style-detail")]')) != 1:
        result['item_name'] = req.xpath('//*[@id="J_Title"]/h3/@data-title').extract_first()

        option_title = req.xpath('//*[@id="J_isku"]/div/dl/dd/ul/@data-property').extract()
        r = []
        for title in option_title:
            j = {}
            j['option_title'] = title
            j['option_details'] = req.xpath('//*[@id="J_isku"]/div/dl/dd/ul[@data-property="{}"]/li/a/span/text()'.format(title)).extract()
            j['option_image_urls'] = [path[17:-29] for path in req.xpath('//*[@id="J_isku"]/div/dl/dd/ul[@data-property="{}"]/li/a/@style'.format(title)).extract()]
            r.append(j)

        result['options'] = r
        
    return result

## T-mall or Taobao

In [ ]:
if "tmall" in current_url:
    result = tmall_crawler(itemid, driver)
else:
    result = taobao_crawler(itemid, driver)

## tmall crawler 만들기

## options

In [160]:
option_path = driver.find_elements_by_xpath('//*[@id="J_DetailMeta"]/div/div/div/div[@class="tb-key"]/div[@class="tb-skin"]/div[@class="tb-sku"]/dl[contains(@class, "tm-sale-prop") and contains(@class, "tb-prop") and contains(@class, "tm-clear")]')
option_ls = []
for path in option_path:
    option = {}
    test['option_title'] = path.find_element_by_xpath('dt').text
    test['oprion_detail'] = path.find_element_by_xpath('dd/ul').text.split('\n')
    image_elements = path.find_elements_by_xpath('dd/ul[contains(@class, "tb-img")]/li/a')
    test['option_image_urls'] = [element.get_attribute('style')[19:-40] for element in image_elements]
    option_ls.append(option)

In [4]:
def tmall_crawler(itemid, driver):
    import pickle
    # 쿠키 복원
    f = open('cookie_taobao.dat', 'rb') 
    cookies = pickle.load(f) 
    for c in cookies: 
        driver.add_cookie(c)
    driver.get("https://item.taobao.com/item.htm?id={}".format(itemid))
    
    result = {}
    result['item_id'] = itemid
    result['item_title'] = driver.find_element_by_xpath('//*[@id="J_DetailMeta"]/div[1]/div[1]/div/div[1]/h1').text
    result['og_price'] = driver.find_element_by_xpath('//*[@id="J_StrPriceModBox"]/dd/span').text
    result['promo_price'] = driver.find_element_by_xpath('//*[@id="J_PromoPrice"]/dd/div/span').text
    option_path = driver.find_elements_by_xpath('//*[@id="J_DetailMeta"]/div/div/div/div[@class="tb-key"]/div[@class="tb-skin"]/div[@class="tb-sku"]/dl[contains(@class, "tm-sale-prop") and contains(@class, "tb-prop") and contains(@class, "tm-clear")]')
    option_ls = []
    for path in option_path:
        option = {}
        option['option_title'] = path.find_element_by_xpath('dt').text
        option['oprion_detail'] = path.find_element_by_xpath('dd/ul').text.split('\n')
        image_elements = path.find_elements_by_xpath('dd/ul[contains(@class, "tb-img")]/li/a')
        option['option_image_urls'] = [element.get_attribute('style')[19:-40] for element in image_elements]
        option_ls.append(option)
        result['options'] = option_ls
        driver.quit()
    return result


## taobao crawler 만들기

In [6]:
def taobao_crawler(itemid, driver):
    import pickle
    # 쿠키 복원
    f = open('cookie_taobao.dat', 'rb') 
    cookies = pickle.load(f) 
    for c in cookies: 
        driver.add_cookie(c)
    driver.get("https://item.taobao.com/item.htm?id={}".format(itemid))

In [10]:
driver.get("https://item.taobao.com/item.htm?id={}".format(itemid))

In [4]:
itemid = "618710171159"

In [3]:
itemid = "563526727068"

In [4]:
driver.get("https://item.taobao.com/item.htm?id={}".format(itemid))

## 가격

In [62]:
# 할인 가격
try:
    print("할인 가격 : ", driver.find_element_by_css_selector('#J_PromoPriceNum').text)
except:
    print('할인 가격 없음')

할인 가격 :  99.00


In [63]:
print("원가 : ", driver.find_element_by_css_selector('#J_StrPrice > em.tb-rmb-num').text)

원가 :  198.00


## 상품명

In [10]:
import requests
from scrapy.http import TextResponse 
import re
from fake_useragent import UserAgent

In [11]:
ua = UserAgent(verify_ssl=False)
userAgent = ua.random
headers = {
    'User-Agent': userAgent
}

url = "https://item.taobao.com/item.htm?id={}".format(itemid)

# https://stackoverflow.com/questions/32910093/python-requests-gets-tlsv1-alert-internal-err
requests.packages.urllib3.util.ssl_.DEFAULT_CIPHERS = 'DH+AESGCM:ECDH+AES256:DH+AES256:ECDH+AES128:DH+AES:ECDH+HIGH:DH+HIGH:ECDH+3DES:DH+3DES:RSA+AESGCM:RSA+AES:RSA+HIGH:RSA+3DES:!aNULL:!eNULL:!MD5'

response = requests.get(url=url,headers=headers)
req = TextResponse(response.url, body=response.text, encoding="utf-8")

In [ ]:
[contains(@class, 'Test')

In [178]:
req.xpath('//div[contains(@classs, "tm-style-detail")]')

[]

In [12]:
# 제품 이름
print("상품명 : ", req.xpath('//*[@id="J_Title"]/h3/@data-title').extract_first())

상품명 :  头层真皮豆豆鞋女春秋夏超级透气软底一脚蹬2020新款孕妇平底单鞋


In [13]:
# 옵션타이틀
option_title = req.xpath('//*[@id="J_isku"]/div/dl/dd/ul/@data-property').extract()

In [14]:
# 옵션 내용
req.xpath('//*[@id="J_isku"]/div/dl/dd/ul[@data-property="{}"]/li/a/span/text()'.format(option_title[0])).extract()

['35', '36', '37', '38', '39', '40', '41', '42']

In [73]:
option_title

['尺码', '颜色分类']

In [106]:
r = []
for title in option_title:
    j = {}
    j['option_title'] = title
    j['options'] = req.xpath('//*[@id="J_isku"]/div/dl/dd/ul[@data-property="{}"]/li/a/span/text()'.format(title)).extract()
    j['image_urls'] = [path[17:-29] for path in req.xpath('//*[@id="J_isku"]/div/dl/dd/ul[@data-property="{}"]/li/a/@style'.format(title)).extract()]
    r.append(j)

# TextResponse

In [1]:
import requests
from bs4 import BeautifulSoup
from scrapy.http import TextResponse 
import re
from fake_useragent import UserAgent

In [255]:
ua = UserAgent(verify_ssl=False)
userAgent = ua.random

In [187]:
# https://stackoverflow.com/questions/32910093/python-requests-gets-tlsv1-alert-internal-err
requests.packages.urllib3.util.ssl_.DEFAULT_CIPHERS = 'DH+AESGCM:ECDH+AES256:DH+AES256:ECDH+AES128:DH+AES:ECDH+HIGH:DH+HIGH:ECDH+3DES:DH+3DES:RSA+AESGCM:RSA+AES:RSA+HIGH:RSA+3DES:!aNULL:!eNULL:!MD5'

In [208]:
test_url = "https://item.taobao.com/item.htm?id=562911697499"
headers = {
    'User-Agent': userAgent}
response = requests.get(url=test_url,headers=headers)

In [209]:
response

<Response [200]>

In [210]:
req = TextResponse(response.url, body=response.text, encoding="utf-8")

In [211]:
req.xpath('//*[@id="J_PromoPrice"]').extract()

['<li id="J_PromoPrice" class="tb-detail-price tb-promo-price tb-clear tb-hidden">\n    <span class="tb-property-type">淘宝价</span>\n    <div class="tb-property-cont">\n        <div id="J_Promo" class="tb-promo-mod">\n            <div id="J_PromoHd" class="tb-promo-hd tb-promo-item"></div>\n            <div id="J_PromoBd" class="tb-promo-bd"></div>\n        </div>\n    </div>\n</li>']

In [212]:
# 제품 이름
req.xpath('//*[@id="J_Title"]/h3/@data-title').extract_first()

'岛上定制新款春秋单鞋低跟英伦真皮女鞋粗跟圆头低帮鞋复古小皮鞋'

In [213]:
# 원가
req.xpath('//*[@id="J_StrPrice"]/em[@class="tb-rmb-num"]/text()').extract_first()

'580.00'

In [202]:
# 할인 가격
req2.xpath('//*[@id="J_PromoHd"]')

[<Selector xpath='//*[@id="J_PromoHd"]' data='<div id="J_PromoHd" class="tb-promo-h...'>]

In [124]:
# 옵션타이틀
option_title = req.xpath('//*[@id="J_isku"]/div/dl/dd/ul/@data-property').extract()

In [144]:
# 옵션 내용
req.xpath('//*[@id="J_isku"]/div/dl/dd/ul[@data-property="{}"]/li/a/span/text()'.format(option_title[0])).extract()

['33', '34', '35', '36', '37', '38', '39', '40']

In [109]:
r = []
for title in option_title:
    j = {}
    j['option_title'] = title
    j['options'] = req.xpath('//*[@id="J_isku"]/div/dl/dd/ul[@data-property="{}"]/li/a/span/text()'.format(title)).extract()
    j['images'] = [path[17:-19] for path in req.xpath('//*[@id="J_isku"]/div/dl/dd/ul[@data-property="{}"]/li/a/@style'.format(title)).extract()]
    r.append(j)

In [110]:
r

[{'option_title': '尺码',
  'options': ['37', '38', '39', '40', '41', '42', '43', '44', '45'],
  'images': []},
 {'option_title': '颜色分类',
  'options': ['黑色皮里', '蓝色皮里', '黑色加绒', '蓝色加绒'],
  'images': ['gd4.alicdn.com/imgextra/i3/169373975/TB2Gp62iRTH8KJjy0FiXXcRsXXa_!!169373975.jpg_30x30.jpg',
   'gd2.alicdn.com/imgextra/i4/169373975/TB2YN0DdgjN8KJjSZFCXXb3GpXa_!!169373975.jpg_30x30.jpg',
   'gd1.alicdn.com/imgextra/i2/169373975/O1CN011fEapNqf2YxZby4_!!169373975.jpg_30x30.jpg',
   'gd4.alicdn.com/imgextra/i1/169373975/O1CN011fEapMBnyxZlrm2_!!169373975.jpg_30x30.jpg']}]